In [14]:
import pandas as pd
import numpy as np
import boto3
import psycopg2
import configparser

In [15]:
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

In [16]:
RDS_HOST = 'tienda-transactional.cjykcg6gmirx.us-east-2.rds.amazonaws.com'
print(RDS_HOST)

tienda-transactional.cjykcg6gmirx.us-east-2.rds.amazonaws.com


In [17]:
postgres_driver = f"""postgresql://{config.get('TRANSACC', 'DB_USER')}:{config.get('TRANSACC', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('TRANSACC', 'DB_PORT')}/{config.get('TRANSACC', 'DB_NAME')}"""

#### Creamos Dimensión de Personas


In [18]:
sql_query = 'SELECT * FROM persona;'
df_personas = pd.read_sql(sql_query, postgres_driver)
df_personas.head()

,idpersona,tipo_persona,nombre,tipo_documento,num_documento,direccion,telefono,email
0,1536244591,empleado,Dillon Frost,Pasaporte,162945912451763,"440 Megan Manors\nSarahstad, SC 64628",38089217,ashley36@hotmail.com
1,1268532496,cliente,Matthew Richards,Pasaporte,155290750418394,"627 Jean Hill Suite 778\nWest Tanyaview, WI 52390",36755108,josephacevedo@hotmail.com
2,1968500916,empleado,Derek Cruz,DNI,193686991975379,"055 Cabrera Points Suite 960\nCindyhaven, WV 4...",36555616,aguirresandra@yahoo.com
3,1109002691,cliente,Ms. Susan Freeman,DNI,158855813307991,"71873 Massey Garden Apt. 052\nSouth Sarah, MO ...",35166718,bbaker@hotmail.com
4,1085777313,proveedor,Bethany Woods,Pasaporte,127352387081821,"91283 Richard Meadow\nEast Tonya, MH 15417",34275114,krystalbarber@hotmail.com


In [19]:
sql_query = 'SELECT * FROM ingreso;'
df_ingreso = pd.read_sql(sql_query, postgres_driver)
df_ingreso.head()

,idingreso,idproveedor,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado
0,0,1735433711,5674321,Nota de Crédito,855596,None,2010-06-26,37.16,55055.88,Completada
1,1,1173940292,1361098,Recibo,938348,None,2010-06-26,76.82,84670.31,Cancelada
2,2,1469297972,5417720,Boleta de Venta,561283,None,2010-06-26,28.48,14607.33,Completada
3,3,1516543853,4719555,Factura,232494,None,2010-06-26,90.64,75219.66,Completada
4,4,1654902231,1729649,Nota de Débito,139725,None,2010-06-26,87.98,89359.46,Completada


In [20]:
df_ingreso = df_ingreso.rename(columns={'idproveedor': 'idpersona'})

In [21]:
dimPersonas = df_personas.merge(df_ingreso, how='inner', on='idpersona')
dimPersonas.drop(['idingreso', 'idusuario'], axis=1,  inplace=True)
dimPersonas.head()

,idpersona,tipo_persona,nombre,tipo_documento,num_documento,direccion,telefono,email,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado
0,1536244591,empleado,Dillon Frost,Pasaporte,162945912451763,"440 Megan Manors\nSarahstad, SC 64628",38089217,ashley36@hotmail.com,Ticket de Venta,897809,None,2010-06-26,98.83,6448.96,Cancelada
1,1109002691,cliente,Ms. Susan Freeman,DNI,158855813307991,"71873 Massey Garden Apt. 052\nSouth Sarah, MO ...",35166718,bbaker@hotmail.com,Nota de Crédito,568942,None,2010-06-26,93.41,70995.04,Completada
2,1109002691,cliente,Ms. Susan Freeman,DNI,158855813307991,"71873 Massey Garden Apt. 052\nSouth Sarah, MO ...",35166718,bbaker@hotmail.com,Recibo,204749,None,2010-06-26,65.68,1792.07,Completada
3,1109002691,cliente,Ms. Susan Freeman,DNI,158855813307991,"71873 Massey Garden Apt. 052\nSouth Sarah, MO ...",35166718,bbaker@hotmail.com,Factura,104596,None,2010-06-26,81.81,17411.70,Cancelada
4,1926261339,proveedor,Brooke Chang,Pasaporte,185501290907297,"74501 Johnson Ramp\nNathanview, TX 62418",38326477,daniellelane@hotmail.com,Nota de Débito,110259,None,2010-06-26,39.05,67167.20,Completada


#### Dimensión de Usuario


In [22]:
sql_query = 'SELECT * FROM usuario;'
df_usuario = pd.read_sql(sql_query, postgres_driver)
df_usuario.head()

,idusuario,idrol,nombre,tipo_documento,num_documento,direccion,telefono,email,clave,estado
0,3904527,10002,Jasmine Fernandez,Pasaporte,194841978531573,"59774 Smith Light Suite 369\nKellyborough, IN ...",32236467,samuel40@gmail.com,"[b'b', b'9', b'2', b'3', b'5', b'1', b'a', b'6...",1
1,5505199,10003,Bruce Carlson,Pasaporte,177072056805478,"586 Zachary Greens\nLake Tara, VT 49975",39516004,ashley82@hotmail.com,"[b'a', b'8', b'1', b'2', b'8', b'c', b'6', b'4...",0
2,3789780,10002,April Conley,NIE,151473726666828,"461 Mcfarland Inlet\nAnthonytown, MI 83513",35858914,usullivan@gmail.com,"[b'a', b'0', b'4', b'6', b'9', b'e', b'4', b'6...",1
3,9987706,10005,Kim Thompson,Pasaporte,115782873172307,"4746 Christopher Crossroad\nRaymondside, PA 96302",32456187,carolcooper@yahoo.com,"[b'1', b'1', b'a', b'e', b'd', b'6', b'b', b'1...",0
4,2607521,10003,Krista Dorsey,NIE,157142427429673,"60309 Richards Loop Apt. 279\nNew Kyle, AK 56872",32818491,eyoung@yahoo.com,"[b'0', b'a', b'8', b'2', b'4', b'3', b'6', b'b...",0


In [23]:
sql_query = 'SELECT * FROM rol;'
df_rol = pd.read_sql(sql_query, postgres_driver)
df_rol.head()

,idrol,nombre,descripcion,estado
0,10001,Administrador,Este rol tiene acceso completo al sistema y pu...,1
1,10002,Moderador,Este rol tiene permisos para moderar contenido...,1
2,10003,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1
3,10004,Soporte Técnico,Este rol proporciona soporte técnico y tiene a...,1
4,10005,Invitado,Este rol tiene acceso limitado y solo puede ve...,1


In [24]:
df_usuario = df_usuario.merge(df_rol, how='inner', on='idrol')
dimUsuario = df_usuario.drop(['idrol'], axis=1)
dimUsuario.head()

,idusuario,nombre_x,tipo_documento,num_documento,direccion,telefono,email,clave,estado_x,nombre_y,descripcion,estado_y
0,3904527,Jasmine Fernandez,Pasaporte,194841978531573,"59774 Smith Light Suite 369\nKellyborough, IN ...",32236467,samuel40@gmail.com,"[b'b', b'9', b'2', b'3', b'5', b'1', b'a', b'6...",1,Moderador,Este rol tiene permisos para moderar contenido...,1
1,3789780,April Conley,NIE,151473726666828,"461 Mcfarland Inlet\nAnthonytown, MI 83513",35858914,usullivan@gmail.com,"[b'a', b'0', b'4', b'6', b'9', b'e', b'4', b'6...",1,Moderador,Este rol tiene permisos para moderar contenido...,1
2,8456118,Kayla Nelson,NIE,125154229533610,"7324 Parker Crossroad\nSouth Susanburgh, WI 74012",36816954,halljessica@yahoo.com,"[b'3', b'9', b'4', b'6', b'7', b'3', b'3', b'6...",1,Moderador,Este rol tiene permisos para moderar contenido...,1
3,7942931,Emily Walls,NIE,175736864841378,"238 Townsend Mills Apt. 616\nRyanmouth, IA 39563",33456230,carrillodebra@gmail.com,"[b'8', b'f', b'3', b'a', b'a', b'6', b'b', b'5...",0,Moderador,Este rol tiene permisos para moderar contenido...,1
4,1546258,Andrea Mcdonald MD,DNI,191091570555321,"694 Rodriguez Green\nAshleyville, KY 32102",39601493,johnbolton@hotmail.com,"[b'9', b'f', b'4', b'7', b'5', b'e', b'd', b'e...",0,Moderador,Este rol tiene permisos para moderar contenido...,1


#### Dimensión Artículo
